<a href="https://colab.research.google.com/github/Mohitg06/pneumoscan-Lab-PPE/blob/master/Pneumoscan_Lab_PPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initializing a model using AWS SageMaker Python API

In [0]:
import json 
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
sess = sage.Session()
role = get_execution_role()
region=sess.boto_region_name

In [0]:
from utils import get_model_package_arn

model_package_arn = get_model_package_arn(sess.boto_region_name)

model = sage.ModelPackage(
    role=role,
    model_package_arn=model_package_arn)

2. Generating Prediction using RealTimePredictor
Creating a new endpoint

In [0]:
endpoint_name='ppe-lab-model-endpoint'

model.deploy(initial_instance_count=1, instance_type='ml.m4.2xlarge', endpoint_name=endpoint_name)

In [0]:
#Define a generic image predictor wrapper which accepts endpoint & session object, and returns a predictor wrapper
def image_predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type='image/jpeg')

#### Generating predictions

In [0]:
predictor = sage.predictor.RealTimePredictor(
    endpoint_name,
    sagemaker_session=sess, 
    content_type="image/jpeg"
)

In [0]:
with os.scandir('data/input/') as entries:
    for entry in entries:
        with open(entry, 'rb') as image:
            f = image.read()
            image_bytes = bytearray(f)       
        prediction_result_image = predictor.predict(image_bytes).decode("utf-8")

In [0]:
import json

result_image = json.loads(prediction_result)
result_image

Analyze Video

In [0]:
from IPython.display import HTML
video_path='./video/video.mp4'

HTML('<iframe width="560" height="315" src="'+video_path+'?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

In [0]:
capture = cv2.VideoCapture(video_path) 

#Get number of frames from the video.
framecount = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

#Take snapshot every 3 second(s)
num_seconds = 3

skip_frames=capture.get(cv2.CAP_PROP_FPS)*num_seconds

num_snapshots=int(framecount/skip_frames)

#bucket = sess.boto_session.resource('s3').Bucket(sess.default_bucket())

#For this experiment, we extract an image every second so that we can utilize the endpoints more efficiently.
for i in range(num_snapshots):
    flag, frame = capture.read()
    if flag:
        path = './video/snapshots/snapshot' + str(i) + '.jpg'
        print ('Creating snapshot on path - ' + path) 
        cv2.imwrite(path, frame) 
        capture.set(cv2.CAP_PROP_POS_FRAMES, ((i+1)*skip_frames))
capture.release()

In [0]:
with os.scandir('video/snapshots/') as entries:
    for entry in entries:
        with open(entry, 'rb') as image:
            f = image.read()
            image_bytes = bytearray(f)
        prediction_result = predictor.predict(image_bytes).decode("utf-8")
   
    



In [0]:
import json

result_video_frame = json.loads(prediction_result_frame)
result_video_frame

In [0]:
sess.delete_endpoint(predictor.endpoint)